<a href="https://colab.research.google.com/github/volkfox/text_style_transfer/blob/master/RPS_archive_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AWS p3.xlarge instance (Nvidia V100), notebook v6 
#



This jupyter sheet is an AWS-based RPS generator. 
For it to work, runtime has to be supported by GPU with sufficient memory (does not happen often on free tier). Recommended v100 or faster.

In [ ]:
# you need 2 files to run the cells:
# Nabokov-all.txt (or other style source for training, 4MB+ in size)
# donor.csv (with columns 'text' and 'author' for content input)
 

In [ ]:
# @kernel-restart
# Ayne Rand "Atlas shrugged" short 1

input = """She sat at the window of the train, her head thrown back, one leg stretched across to the empty seat
before her. The window frame trembled with the speed of the motion, the pane hung over empty
darkness, and dots of light slashed across the glass as luminous streaks, once in a while.

Her leg, sculptured by the tight sheen of the stocking, its long line running straight, over an arched
instep, to the tip of a foot in a high-heeled pump, had a feminine elegance that seemed out of place in
the dusty train car and oddly incongruous with the rest of her. She wore a battered camel's hair coat
that had been expensive, wrapped shapelessly about her slender, nervous body. The coat collar was
raised to the slanting brim of her hat. A sweep of brown hair fell back, almost touching the line of her
shoulders. Her face was made of angular planes, the shape of her mouth clear-cut, a sensual mouth
held closed with inflexible precision. She kept her hands in the coat pockets, her posture taut, as if
she resented immobility, and unfeminine, as if she were unconscious of her own body and that it was
a woman's body. She sat listening to the music. It was a symphony of triumph. The notes flowed up,
they spoke of rising and they were the rising itself, they were the essence and the form of upward
motion, they seemed to embody every human act and thought that had ascent as its motive. It was a
sunburst of sound, breaking out of hiding and spreading open. It had the freedom of release and the
tension of purpose. It swept space clean, and left nothing but the joy of an unobstructed effort. Only a
faint echo within the sounds spoke of that from which the music had escaped, but spoke in laughing
astonishment at the discovery that there was no ugliness or pain, and there never had had to be. It was
the song of an immense deliverance."""


In [ ]:
# one-time
# install dependencies if needed 

!pip install sacremoses
!pip install gpt-2-simple 
!pip install nltk
!pip install wrapt --upgrade --ignore-installed
!pip install --upgrade tensorflow-gpu==1.14
!pip install tensorflow-hub 

# instructions to enable 774M model:
#
# vi /home/ubuntu/anaconda3/lib/python3.7/site-packages/gpt_2_simple/gpt_2.py
# comment out line: assert model_name not in ['774M', '1558M']

In [1]:
# @kernel restart

import sys, getopt
import gc
from pathlib import Path
from sacremoses import MosesDetokenizer
import gpt_2_simple as gpt2
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd 
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [ ]:
# one-time
# download tokenizer file
nltk.download('punkt')

In [2]:
# @kernel restart
# GPT-2 released three models
#model_name="117M"
#model_name="774M"

model_name="345M"

# hyperparameters
minsimilarity = 0.68 # rejection threshold
# lexeme bounds
minwords = 2         # min lexeme length
softmaxwords = 4     # min lexeme ending in -and-
maxwords = 7        # max lexeme without a clear end
# generator parameters
temperature = 1.0    # generator madness
seedLexemes = 10     # how many lexemes to put in a prior
nsamples = 10000      # number of samples generated to choose from


bestLexemes = []
output = ""

# this is your style for GPT-2 finetuning
style = "Nabokov-All.txt"

# content input and output
generator_name = 'Trained-21K-Dumas'
outfile = generator_name+"_"+model_name+"_"+str(nsamples)+"_"+style
sfile = outfile+".stats"

# get around resource exhaustion for embeddings
config = tf.ConfigProto(
        device_count = {'GPU': 0}
        #device_count = {'GPU': 1}
    )

# read text data to transform
data = pd.read_csv("donor.csv")
data = data[data['author']=='Dumas']

In [3]:
# @kernel restart
# init tokenizer and sentence embedder
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
dt = MosesDetokenizer()

# Google Sentence encoder v2 appears worse than V1
module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"
#module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
embed = hub.Module(module_url)
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# download GPT-2 model (only needed once)
gpt2.download_gpt2(model_name=model_name)


In [4]:
# TRAINING
# set number of steps for finetuning or load the pretrained model
# if a model is already mounted, no training required

sess = gpt2.start_tf_sess()
# load an existing, fine-tuned model
# gpt2.load_gpt2(sess)
#
# finetuning frequently fails in collab based on resource exhaustion
# Good training duration results in avg loss between 0.2 and 0.9 (~20K training iterations on 345M model)
#
# GPT-2 can generate coherent output even on low finetuning, but with generic style (default)

# fine-tune from scratch

#gpt2.finetune(sess, style, model_name=model_name, restore_from = 'latest', overwrite=True, steps=8000)

# continue training from a saved model

gpt2.finetune(sess, style, model_name=model_name, overwrite=True, steps=4000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Loading checkpoint checkpoint/run1/model-17000
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from checkpoint/run1/model-17000


...

Loading dataset...


100%|██████████| 1/1 [00:15<00:00, 15.17s/it]


dataset has 2363594 tokens
Training...
Saving checkpoint/run1/model-17000
Saving checkpoint/run1/model-17000
======== SAMPLE 1 ========
 obvious in the very beginning of life to let such a grand, and wise, and just and merciful Godhead work for a moment, so that all might be revealed — which is what good faith is all about, what respect, which is everything. We mustn’t make the first bargain yet, my sweet Joan. You might say I had my little fillylike [kissing the girl on the cheek] today. Now let me tell you, there are some things, Betty, that seem awfully silly. Piffle, freak, flap. And what about braces and wings and [breathing the same barely audible] "birdies?"

“No, no, Joan, you always find a way. All right, all right, we can't afford them all.”

“Well, Betty, we thought you might like braces and wings. Actually, they are making us even. It is we who bought them, not them.”

“And what about a little plastic surgery?”

“That is exactly what we were afraid of. So let’s not splutter

Instructions for updating:
Use standard file APIs to delete files with this prefix.


======== SAMPLE 1 ========
 she had had an inkling that something had happened, but she only realized it later because of a kind of dream feeling she got from Luzhin as he reclined in a deep armchair, apparently asleep.

The next day there was still another meeting, and this one was particularly painful for Fyodor because he had not the faintest notion how to begin. The vague topic with which he settled after having read his author's work did not come up, as had been his intention at the very beginning of their meetings; he could not get to this nor could he find out how exactly the money he had sent was being distributed; among the documents he produced to show how the money had been spent he produced an astonishing document, of a howitzerslot, entitled The Protector, and this document, as he later explained to Zina, was the only piece of genuine, absolutely reliable information he finally needed for his decision.

The idea that his life was but a series of yellow corridors till the o

In [5]:
# @every kernel restart
# take a list of unstructured expansions and tidy them up
# works around the following problem: the generated piece may include seed or not
#
# expansion formation rules: seed phrase deleted, truncation at first end 
# of sentence, the last comma, or allow to run off unclipped otherwise
# 

def cleaned(expansions, seed):

   seedbag = word_tokenize(seed)
   cutseed = dt.detokenize(seedbag[1:])
   symlen = len(cutseed)
   clean = []
   expansion = ""
   for exp in expansions:
      pos = exp.find(cutseed)
      if pos > 0:
         expansion = exp[pos+symlen:].strip()
      else:
         # sometimes search is thrown off w/special characters
         # heuristics here assumes expansion larger than seed must be trimmed
         expbag = word_tokenize(exp)
         if len(expbag) > len(seedbag):
            expansion = dt.detokenize(expbag[len(seedbag):])
         else:
            
         # all attempts to identify seed failed, assume generator gave no seed
            expansion = exp
            
      # try to end with a period
      pos = expansion.find(".")
      if pos > 0:
         expansion = expansion[:pos]+"."
      else: # or exclamation
         pos = expansion[::-1].find("!")
         if pos > 0:
            expansion = expansion[:-pos]
         else: # or comma
            pos = expansion[::-1].find(",")
            if pos > 0:
               expansion = expansion[:-pos]
      clean.append(expansion)
      
   return clean

In [ ]:
seed = """. he has an incredible appetite for her- -- and she, of course, did not take any interest in him at all! He has become quite a simpleminded man. And I think that, overall- Such an admirer of her performance on the stage as I was, and of her lyrical, and of her'northern music! "I can well remember," he said at one time listen to her for ever. Oh! and I had almost forgotten one idea that """
expansions = [""". he has an incredible appetite for her- -- and she, of course, did not take any interest in him at all! He has become quite a simpleminded man. And I think that, overall- Such an admirer of her performance on the stage as I was, and of her lyrical, and of her'northern music! "I can well remember," he said at one time listen to her for ever. Oh! and I had almost forgotten one idea that everything is fine"""]



In [ ]:
cleaned(expansions, seed)


In [ ]:
seedbag = word_tokenize(seed)
cutseed = dt.detokenize(seedbag[1:])
symlen = len(cutseed)
cleaned = []
expansion = ""
pos = expansions[0].find(cutseed)

In [ ]:
len(seedbag)

In [6]:
# @every kernel restart
# define the next piece to replace in the donor text
def generateLexeme(sent):

      if sent == "":
         return

      words = word_tokenize(sent)

      lexeme = ""
      count = 0

      if len(words) <= maxwords:
         processLexeme(sent)

      else:
         for word in words:
            count += 1
            if (word in [";", ",","-","--"] and count > minwords) or (word in ["and", "for", "at"] and count > softmaxwords) or count > maxwords:
               if (len(words)-count) >  minwords:
                  processLexeme(dt.detokenize(words[:count]))
                  sentence = dt.detokenize(words[count:])
                  generateLexeme(sentence)
               else:
                  processLexeme(dt.detokenize(words))

               break

In [7]:
# @every kernel restart
# main RPS routine: extract a lexeme, generate candidates, replace with a best one
def processLexeme(sent):

   # always start from the first phrase
   if not bestLexemes:
      bestLexemes.append(sent)
      print("Leading output with seed phrase: {}".format(sent))
      return
    
   seed = dt.detokenize(bestLexemes[-seedLexemes:])
   print("seed: \"{}\"".format(seed))
   print("\t  processing lexeme: \"{}\"".format(sent))
   
   expansions = gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, return_as_list=True, include_prefix=False)
   #expansions = gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, truncate=".", return_as_list=True, include_prefix=False)
   #expansions += gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, truncate=",", return_as_list=True, include_prefix=False)
   expansions = cleaned(expansions, seed)
   #print("  expansion set: {}".format(expansions))
  
   messages = [sent]+expansions
   bestLexeme = sent
  
   gc.collect()

   with tf.Session(config=config) as session:
      session.run(tf.global_variables_initializer())
      session.run(tf.tables_initializer())
      message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})

      corr = np.inner(message_embeddings_, message_embeddings_)
      embeddings = corr[0,1:]
      #print("embeddings: {}".format(embeddings))
      
      bestIndex = np.argmax(embeddings)
      
      if embeddings[bestIndex] >  minsimilarity:
        
         bestLexeme = expansions[bestIndex]
            
         # stats       
         # calculate running average for accepted scores

         acceptedLexemes["accepted"] = acceptedLexemes["accepted"]+1
         acceptedLexemes["candidate_percent"].append(np.sum(embeddings > minsimilarity)/nsamples)
            
         # join expansion with future lexeme smoothly
        
         originalEnding = sent[-1]
         end = originalEnding if originalEnding in [".",",", ";", "-", "--", "?", "!", "...", "and","for", "at"] else ""
        
         lexemeEnding = bestLexeme[-1]
         
         # replace punctuation at end of lexeme if original punctuation existed
         # if donor sentence not ending yet, ignore the expansion markers
         if lexemeEnding in [".", ",", ";", "?", "!", "and", "or", "for", "at"]:
            bestLexeme = bestLexeme[:-1]+end
         # trying a plug from the content. Removing this line allows a lot of freedom to style engine.
         else:
            bestLexeme = bestLexeme+end
            
      else:
         acceptedLexemes["rejected"] = acceptedLexemes["rejected"]+1  
        
   print("  expanding with: \"{}\" bestscore: {} bestphrase: {}".format(bestLexeme, embeddings[bestIndex], expansions[bestIndex]))
   bestLexemes.append(bestLexeme)

   output = dt.detokenize(bestLexemes)
   Path(outputfile).write_text(output)
    
   #stats 
   acceptedLexemes["total_score"] = acceptedLexemes["total_score"]+embeddings[bestIndex]
   print(acceptedLexemes)
   Path(statsfile).write_text(str(acceptedLexemes))
   

In [ ]:
# launch RPS from here.
# 

transformed = []

for ind, row in data.iterrows():
    
    outputfile =  outfile + str(ind)
    statsfile = sfile + str(ind)
    input = row['text']
    
    bestLexemes = []
    acceptedLexemes = {"accepted":0, "rejected":0, "total_score": 0, "candidate_percent":[]}

    sentences = tokenizer.tokenize(input)

    for sent in sentences:
       print("Input sentence: {}".format(sent))
       generateLexeme(sent)

    output = dt.detokenize(bestLexemes)
    #Path(outputfile).write_text(output)

    print("\n")
    print(output)
    print("\n")
    print("------------------------------------------------------------")
    print(input)
    transformed.append(output)
    np.savetxt(outfile+".csv", transformed, delimiter=",", fmt='%s')


Input sentence:  of Jane Fairfax!
Leading output with seed phrase:  of Jane Fairfax!
Input sentence: The interest he takes in her--his anxiety about her health--his concern that she should have no happier prospect!
seed: "of Jane Fairfax!"
	  processing lexeme: "The interest he takes in her --"
  expanding with: "The interest he takes in her --" bestscore: 0.6534385681152344 bestphrase: (“She rolls with the others”).
{'accepted': 0, 'rejected': 1, 'total_score': 0.6534385681152344, 'candidate_percent': []}
seed: "of Jane Fairfax! The interest he takes in her --"
	  processing lexeme: "his anxiety about her health --"
  expanding with: "this interest in her, which causes his mind to overload-" bestscore: 0.6857276558876038 bestphrase: this interest in her, which causes his mind to overload,
{'accepted': 1, 'rejected': 1, 'total_score': 1.3391662240028381, 'candidate_percent': [0.0001]}
seed: "of Jane Fairfax! The interest he takes in her -- this interest in her, which causes his mind to

In [ ]:
print(bestLexemes)

In [ ]:
from tensorflow.contrib.memory_stats.python.ops.memory_stats_ops import BytesInUse
with tf.device('/device:GPU:0'):  # Replace with device you are interested in
  bytes_in_use = BytesInUse()
with tf.Session() as sess:
  print(sess.run(bytes_in_use))